In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

### Train data

In [2]:
df_train = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_1/clean_training_data', sep=',')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_1/clean_test_data', sep=',')

In [4]:
# For convenience, it is considered a unique dataframe with different keys. 
features = pd.concat([df_train,df_test], keys=['train','test'])

In [5]:
# Numerical features are scaled
features_num = features[['TotalPackets','TotalBytes','SourceBytes','PacketsSec','TotalBytesSec','SourceBytesSec']]

In [8]:
features_num.replace([np.inf,np.nan], -1,inplace=True)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [9]:
features_num.isna().any()

TotalPackets      False
TotalBytes        False
SourceBytes       False
PacketsSec        False
TotalBytesSec     False
SourceBytesSec    False
dtype: bool

In [10]:
scaler = StandardScaler()
scaler.fit(features_num.loc['train'])

StandardScaler()

In [11]:
df_train_num = pd.DataFrame(scaler.transform(features_num.loc['train']), columns=features_num.columns,
                                   index=features_num.loc['train'].index)

In [12]:
df_test_num = pd.DataFrame(scaler.transform(features_num.loc['test']), columns=features_num.columns,
                                   index=features_num.loc['test'].index)

In [13]:
# LabelEncoder is applied to categorical features
features_cat = features[['Protocol', 'SrcIPAddress', 'SrcPort','Direction',
                         'DestIPAddress', 'DestPort', 'State', 'SrcTypeService','DestTypeService']]

In [14]:
encoders = {}

for column in features_cat:
    print(column)
    encoders[column] = preprocessing.LabelEncoder()
    encoders[column].fit(features_cat[column].astype(str))

Protocol
SrcIPAddress
SrcPort
Direction
DestIPAddress
DestPort
State
SrcTypeService
DestTypeService


In [15]:
df_test_cat = features_cat.loc['test'].apply(lambda x: encoders[x.name].transform(features_cat.loc['test'][x.name].astype(str)))


In [16]:
df_train_cat = features_cat.loc['train'].apply(lambda x: encoders[x.name].transform(features_cat.loc['train'][x.name].astype(str)))


In [17]:
# Finally is concatenated numerical and categorical data
train_data = pd.concat([df_train_num, df_train_cat], axis=1)

In [18]:
train_data.to_csv('../../ml-data/cyberattack_detection/2_processed_data/A1_train_data.csv', sep=',', index=False)

In [19]:
# Finally is concatenated numerical and categorical data
test_data = pd.concat([df_test_num, df_test_cat], axis=1)

In [20]:
test_data.to_csv('../../ml-data/cyberattack_detection/2_processed_data/A1_test_data.csv', sep=',', index=False)

End